## Policy Gradient
To maximize the objective function $J(\theta)$
    $$
    \underset{\theta}{\operatorname{arg max}}J(\theta) = \underset{\theta}{\operatorname{arg max}} \mathbb{E}_{s \sim d_\pi}\left [ \right ] 
    $$
we can use the policy gradient theorem:
$$
\nabla_\theta J(\theta)  = \mathbb{E}_{s \sim d_\pi}\left [\mathbb{E}_{a \sim \pi_{\theta}(\cdot \vert s)} [Q_\pi(s, a) \nabla_\theta \ln \pi_\theta(a \vert s)] \right ] 
$$
Sample from $a \sim \pi_{\theta}(\cdot \vert s)$ and a monte carlo estimation of $\nabla_\theta J(\theta)$ is:
$$
\boldsymbol{g}_\theta(s, a) \doteq Q_{\pi_\theta}(s, a) \nabla_\theta \ln \pi_\theta(a \vert s)
$$

## Generalization Advantage Estimation(GAE)
$$
\hat{A}_t^{GAE(\gamma, \lambda)} = \sum_{l=0}^{\infty} (\gamma \lambda)^l \delta_{t+l}
$$
![](./doc/gae_0.png)
![](./doc/gae_1.png)

![](./doc/gae_2.png)


## The TRPO formula
Importance sampling ratio:
$$
\underset{\theta}{\operatorname{arg max}} \mathbb{E}_{s}\left [\mathbb{E}_{a \sim \pi_{\theta_\text{old}}(\cdot \vert s)} [  \frac{\pi_\theta(a \vert s)}{\pi_{\theta_\text{old}}(a \vert s)} \hat{A}_{\theta_\text{old}}(s, a) ] \right]
$$

## The PPO Algorithm
$$
\underset{\theta}{\operatorname{arg max}} \mathbb{E}_{s}\left [\mathbb{E}_{a \sim \pi_{\theta_\text{old}}(\cdot \vert s)} [\min(\frac{\pi_\theta(a \vert s)}{\pi_{\theta_\text{old}}(a \vert s)} \hat{A}_{\theta_\text{old}}(s, a), \text{clip}(\frac{\pi_\theta(a \vert s)}{\pi_{\theta_\text{old}}(a \vert s)}, 1 - \epsilon, 1 + \epsilon) \hat{A}_{\theta_\text{old}}(s, a))]\right]
$$
